In [ ]:
# 'https://drive.google.com/file/d/1XjNIpICZFUknNQ7Pc9m4aeXr_F310vEO/view?usp=sharing'  dataset link

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1XjNIpICZFUknNQ7Pc9m4aeXr_F310vEO'}) # replace the id with id of file you want to access
downloaded.GetContentFile('data.csv') 

In [ ]:
# import libearies
import numpy as np
import pandas as pd
from collections import Counter 
from datetime import datetime
import re
import itertools

In [ ]:
data=pd.read_csv('data.csv',low_memory=False)#reading the dataset

#### Here I use nested function concept
#### It would have been much simpler if I had permission to use the class

In [ ]:
def selector(char_num):
#     objective :for each categorical columns calculate which have numerical value
#       return :1. characters_only: column which have only character column
#               2. not_available: numerical columns nan values as 'not available'
#               3. charcters_non_characters: data which have both numerical and categorical value in one column
    characters_only=[]
    charcters_non_characters=[]
    not_available=[]
    for col in char_num.columns:#iterate categorical columns
        groups=[]
        for i in data[str(col)].value_counts().index.values:#for each categorical column fond the unique values
            find_no=re.search(r'\d+',i)#match numerical values
            if find_no is None:#if doesnot have numerical values 
                groups.append('char_only')#if find 'char_only' at that place append those values
            else:
                groups.append(find_no.group(0))#if find numerical values append those values
                
        if all(char == 'char_only' for char in groups):#if all data have 'char_only'
            characters_only.append(col)#make that column as character
        #in some numerical columns nan values as 'not available'
        #if we take the unique values of that column 'not availble' will get count for only once
        # algo will find it as charcter and count it only for once
        elif(Counter(groups)['char_only']==1):#column which have only one count  
            replace_item=data[col].value_counts().index.values[0]
            data[col]=data[col].replace(replace_item,np.nan)#replace not available with np.nan
            not_available.append(col)
        #finally data which have both character and numerical value for multiple time will get append to 'charcters_non_characters'list
        else:
            charcters_non_characters.append(col)
    return characters_only,not_available,charcters_non_characters

In [ ]:
def char_chep(col_list):
#     objective :extract numerical data from dataset which are having characters and non characters together and have not avilable columns
#       return 1.extract only ctaegorical column from which column are having categorical and numerical together
#       return 2:extract only numerical column from which column are having categorical and numerical together
    char_dat=[]
    num_dat=[]
    for x in col_list:
        groups=[]
        for i in data[str(x)].value_counts().index.values:#for each column fond the unique values
            # check wheater the columns have values like,eg:'$1,234','100m^2  ','12,345'
            d=re.search(r'(^-?[\$|€|¥|£|A\$|C\$|CHF|元|HK\$|NZ\$|kr|₩|S\$|kr|\$|₹|₽|R|₺|R\$|NT\$|kr|zł|฿|Rp|Ft|Kč|₪|CLP\$|₱|د.إ|COL\$|﷼|RM|L][\d,\.]+$)|(^-?[\d,?\.?]+[a-zA-Z0-9]*$)',i)
            if d is None:#if doesnot have numerical values 
                groups.append('num_none')#if dont match with regex append 'num_none' at that place
            else:
                groups.append(d.group(0))#if match with string append match string at that place
        if any(char == 'num_none' for char in groups):# select only those column if any value doesnot match with regex
            char_dat.append(x)
        else:
            data[x]=data[x].apply(lambda l:re.sub(r'[^\d]+','',str(l)))# substitute the chracter values eg:'$1,234'to 1234,'100m^2'to 100
            data[x]=data[x].replace('',np.nan)#some column already had none values replace to np.nan
            num_dat.append(x)
        
    return char_dat,num_dat

In [ ]:
#inputs: 1.dataset=DataFrame
#        2.max _threshold=if the column have outliers,percent of data you want to keep as outlier
def detect_outlier(dataset,max_threshold=None):
    numerical_data=dataset.select_dtypes(include=np.number)# select numerical columns only
    char_num_data=dataset.select_dtypes(exclude=np.number)# selectcategorical columns only
    char_only_data,num_data1,num_data2=selector(char_num_data)# call selector function
    
    characters_only,numericals_only=char_chep(num_data2+num_data1)# call char chep function input:'not available'+categorical and numerical columns
    characters_only=characters_only+char_only_data
    
    cat_data=data[np.asarray(characters_only)]# final dataframe of characters only
    num_data=pd.concat([data[np.asarray(numericals_only)],numerical_data],axis=1)# final dataframe of numericals only
    
    num_data=num_data.astype('float32')#convert numerical data to float data
    
    # Outlier detection ,method:inter quartile range
    # create an iterator if algo removes more than the max_threshold percent expand inter quartile range
    for i in itertools.count(1):
        strt_time = datetime.now()
        indexes=np.array([])
        for x in num_data.columns:
            mini=num_data[x].min()#minimum value of that column
            maxi=num_data[x].max()#maximum value of that column
            first_quartile=num_data[x].describe()['25%']
            third_quartile=num_data[x].describe()['75%']
            iqr=third_quartile-first_quartile# calculate iqr
            if iqr!=0.0:
                Lower_Bound=(first_quartile - (iqr*1.5*i))
                Upper_Bound=(first_quartile +  (iqr*1.5*i))
                if (Upper_Bound<maxi) and (mini>Lower_Bound):# if bigger values contain outliers
                    indexes=np.concatenate((indexes,np.where(num_data[x]>Upper_Bound)[0])).astype(int)
                elif (Upper_Bound>maxi) and (mini<Lower_Bound):# if smaller values contain outliers
                    indexes=np.concatenate((indexes,np.where(num_data[x]<Lower_Bound)[0])).astype(int)
                elif (Upper_Bound<maxi) and (mini<Lower_Bound):# if both smaller and bigger  values contain outliers
                    indexes=np.concatenate((indexes,np.where((num_data[x]>Upper_Bound) &(num_data[x]<Lower_Bound))[0])).astype(int)
                else:
                    continue
            else:
                continue
        if(len(indexes)<((num_data.shape[0]*max_threshold)/100)):#if numer of otliers are smaller than max_threshold
            num_data.drop(index=indexes,axis=0,inplace=True)# drop otliers
            return num_data
        else:
            end_time = datetime.now()
            print(f'To many outlers ,iter {i} time {end_time-strt_time} ')
            continue

In [ ]:
da=detect_outlier(data,10)
da

To many outlers ,iter 1 time 0:00:00.165030 
To many outlers ,iter 2 time 0:00:00.162147 
To many outlers ,iter 3 time 0:00:00.160226 
To many outlers ,iter 4 time 0:00:00.153455 
To many outlers ,iter 5 time 0:00:00.169630 
To many outlers ,iter 6 time 0:00:00.155800 
To many outlers ,iter 7 time 0:00:00.157549 
To many outlers ,iter 8 time 0:00:00.156863 
To many outlers ,iter 9 time 0:00:00.158155 
To many outlers ,iter 10 time 0:00:00.154059 
To many outlers ,iter 11 time 0:00:00.169643 
To many outlers ,iter 12 time 0:00:00.158674 
To many outlers ,iter 13 time 0:00:00.160466 
To many outlers ,iter 14 time 0:00:00.157245 
To many outlers ,iter 15 time 0:00:00.161735 
To many outlers ,iter 16 time 0:00:00.155169 
To many outlers ,iter 17 time 0:00:00.174014 
To many outlers ,iter 18 time 0:00:00.163138 
To many outlers ,iter 19 time 0:00:00.175789 
To many outlers ,iter 20 time 0:00:00.171254 
To many outlers ,iter 21 time 0:00:00.159576 
To many outlers ,iter 22 time 0:00:00.15372

,Parent Property Id,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type - Gross Floor Area (ft²),ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),District Steam Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),Order,Property Id,DOF Gross Floor Area,Year Built,Number of Buildings - Self-reported,Occupancy,Property GFA - Self-Reported (ft²),Latitude,Longitude,Community Board,Council District,Census Tract
2,28400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4778226.0,152765.0,1924.0,1.0,100.0,231342.0,40.840427,-73.940247,12.0,10.0,251.0
3,28400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4778267.0,891040.0,1971.0,1.0,100.0,1305748.0,40.840748,-73.942856,12.0,10.0,255.0
4,28400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4778288.0,211400.0,1932.0,1.0,100.0,179694.0,40.841560,-73.942528,12.0,10.0,255.0
6,28402.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,4778352.0,245000.0,1932.0,1.0,60.0,245000.0,40.765949,-73.953751,8.0,5.0,124.0
12,2637863.0,20000.0,18800.0,93.0,32.0,NaN,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2047200.0,20472.0,112419264.0,32497552.0,22061.0,5101.0,879.0,15.0,2637863.0,NaN,1903.0,2.0,100.0,432503.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11741,NaN,NaN,NaN,98.0,814.0,87.0,33.0,8.0,1149.0,NaN,NaN,NaN,NaN,NaN,NaN,51796000.0,561689.0,857323.0,2452511.0,23085.0,3112.0,1096.0,14988.0,4940453.0,70645.0,1928.0,1.0,100.0,74177.0,40.696419,-73.992493,2.0,33.0,502.0
11742,NaN,7791.0,NaN,16.0,1093.0,1177.0,53.0,9.0,1608.0,NaN,565800.0,NaN,NaN,NaN,NaN,86168000.0,95350.0,18855088.0,5326727.0,42168.0,4163.0,1534.0,14989.0,4940416.0,93500.0,1926.0,1.0,100.0,101291.0,40.692600,-73.993233,2.0,33.0,7.0
11743,NaN,NaN,NaN,83.0,443.0,439.0,77.0,2.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,17616298.0,18131.0,28188548.0,7968372.0,NaN,NaN,1036.0,14990.0,4628296.0,NaN,1913.0,1.0,95.0,103328.0,NaN,NaN,NaN,NaN,NaN
11744,NaN,NaN,NaN,92.0,70.0,749.0,26.0,7.0,969.0,NaN,NaN,NaN,NaN,NaN,NaN,36935000.0,401626.0,5567599.0,1553193.0,16876.0,278.0,927.0,14991.0,4940464.0,57824.0,1928.0,1.0,100.0,60715.0,40.670727,-73.971748,6.0,39.0,165.0


In [ ]:
# Dataset should have only numerical values
# For convienience I will use the dataframe from outlier detection 
def correlation_finder(dataset):
  
  corr=abs(dataset.corr(method='pearson'))# Find correlation
  corr=corr.unstack().reset_index() # making row and column as separate feature
  corr.columns=['Feature1','Feature2','Correlations']
  remove_self_duplicates=corr[corr.Feature1!=corr.Feature2]# remove those columns which have (corr=1) with same value
  columns_to_drop=remove_self_duplicates[remove_self_duplicates['Correlations']>0.85].sort_values(by='Correlations',ascending=False)[::2].Feature2.unique().tolist()# columns having more than 0.85 correlations
  dataset.drop(columns=columns_to_drop,axis=1,inplace=True) # drop columns
  return dataset

In [ ]:
output=correlation_finder(da)
output

,3rd Largest Property Use Type - Gross Floor Area (ft²),ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),Order,Property Id,DOF Gross Floor Area,Year Built,Number of Buildings - Self-reported,Occupancy,Property GFA - Self-Reported (ft²),Latitude,Longitude,Community Board,Census Tract
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4778226.0,152765.0,1924.0,1.0,100.0,231342.0,40.840427,-73.940247,12.0,251.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4778267.0,891040.0,1971.0,1.0,100.0,1305748.0,40.840748,-73.942856,12.0,255.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4778288.0,211400.0,1932.0,1.0,100.0,179694.0,40.841560,-73.942528,12.0,255.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,4778352.0,245000.0,1932.0,1.0,60.0,245000.0,40.765949,-73.953751,8.0,124.0
12,18800.0,93.0,32.0,NaN,75.0,0.0,NaN,NaN,NaN,NaN,2047200.0,20472.0,112419264.0,32497552.0,22061.0,5101.0,879.0,15.0,2637863.0,NaN,1903.0,2.0,100.0,432503.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11741,NaN,98.0,814.0,87.0,33.0,8.0,1149.0,NaN,NaN,NaN,51796000.0,561689.0,857323.0,2452511.0,23085.0,3112.0,1096.0,14988.0,4940453.0,70645.0,1928.0,1.0,100.0,74177.0,40.696419,-73.992493,2.0,502.0
11742,NaN,16.0,1093.0,1177.0,53.0,9.0,1608.0,565800.0,NaN,NaN,86168000.0,95350.0,18855088.0,5326727.0,42168.0,4163.0,1534.0,14989.0,4940416.0,93500.0,1926.0,1.0,100.0,101291.0,40.692600,-73.993233,2.0,7.0
11743,NaN,83.0,443.0,439.0,77.0,2.0,101.0,NaN,NaN,NaN,17616298.0,18131.0,28188548.0,7968372.0,NaN,NaN,1036.0,14990.0,4628296.0,NaN,1913.0,1.0,95.0,103328.0,NaN,NaN,NaN,NaN
11744,NaN,92.0,70.0,749.0,26.0,7.0,969.0,NaN,NaN,NaN,36935000.0,401626.0,5567599.0,1553193.0,16876.0,278.0,927.0,14991.0,4940464.0,57824.0,1928.0,1.0,100.0,60715.0,40.670727,-73.971748,6.0,165.0
